In [46]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

raw_data = pd.read_csv('../data/hotel_bookings.csv')
raw_data.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [47]:
##Transform / Prepare data
# Convert object types as well as selected fields (agent, company) to string for processing
for col in ('hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'agent', 'company'):
    raw_data[col] = raw_data[col].astype('string')

#Transform Date into Date_time
raw_data['reservation_status_date'] = pd.to_datetime(raw_data['reservation_status_date'])

#convert binary values into true, false
raw_data['is_canceled'] = raw_data['is_canceled'].astype(bool)
raw_data['is_repeated_guest'] = raw_data['is_repeated_guest'].astype(bool)
raw_data['days_in_waiting_list'] = raw_data['days_in_waiting_list'].astype(bool)
raw_data = raw_data.rename(columns={"days_in_waiting_list": "was_in_waiting_list"})
raw_data['required_car_parking_spaces'] = raw_data['required_car_parking_spaces'].astype(bool)
raw_data['total_of_special_requests'] = raw_data['total_of_special_requests'].astype(bool)
raw_data = raw_data.rename(columns={"total_of_special_requests": "has_special_requests"})

raw_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,was_in_waiting_list,customer_type,adr,required_car_parking_spaces,has_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,False,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,False,0,0,C,C,3,No Deposit,<NA>,<NA>,False,Transient,0.0,False,False,Check-Out,2015-07-01
1,Resort Hotel,False,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,False,0,0,C,C,4,No Deposit,<NA>,<NA>,False,Transient,0.0,False,False,Check-Out,2015-07-01
2,Resort Hotel,False,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,False,0,0,A,C,0,No Deposit,<NA>,<NA>,False,Transient,75.0,False,False,Check-Out,2015-07-02
3,Resort Hotel,False,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,False,0,0,A,A,0,No Deposit,304.0,<NA>,False,Transient,75.0,False,False,Check-Out,2015-07-02
4,Resort Hotel,False,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,False,0,0,A,A,0,No Deposit,240.0,<NA>,False,Transient,98.0,False,True,Check-Out,2015-07-03


In [48]:
##Remove Outliers and invalid data
#Remove Adults outliers
raw_data = raw_data.drop(raw_data[(raw_data.adults >= 10)].index)

#Remove data with invalid number of nights
raw_data = raw_data.drop(raw_data[(raw_data.stays_in_weekend_nights == 0) & (raw_data.stays_in_week_nights == 0)].index)

# Remove data with no country assigned
raw_data = raw_data.drop(raw_data[pd.isna(raw_data.country)].index)

# Remove Children & Babies outliers
raw_data = raw_data.drop(raw_data[pd.isna(raw_data.children) | (raw_data.children > 3) | (raw_data.babies > 2)].index)

In [49]:
## Reduce data volume to allow more efficient data processing
def remove_random_rows(reservation_status):
    sample_indizes = raw_data.index[raw_data['reservation_status'] == reservation_status].tolist()
    remove_num = len(sample_indizes) - 1189
    print("remove_num: ", remove_num)
    drop_indices = np.random.choice(sample_indizes, remove_num, replace=False)
    return raw_data.drop(drop_indices)

print(len(raw_data.index))

np.random.seed(10)

raw_data = remove_random_rows('No-Show')
raw_data = remove_random_rows('Check-Out')
raw_data = remove_random_rows('Canceled')

print(len(raw_data.index))


118181
remove_num:  0
remove_num:  72879
remove_num:  41735
3567


In [50]:
##Map and reduce data
#Map Lead Time
for i in raw_data.index:
    if raw_data['lead_time'][i] > 365:
        raw_data['lead_time'][i] = "very-long-term"
    elif raw_data['lead_time'][i] > 180:
        raw_data['lead_time'][i] = "long-term"
    elif raw_data['lead_time'][i] > 90:
        raw_data['lead_time'][i] = "moderate"
    elif raw_data['lead_time'][i] > 30:
        raw_data['lead_time'][i] = "short-term"
    else:
        raw_data['lead_time'][i] = "very-short-term"

raw_data['lead_time'] = raw_data['lead_time'].astype('string')

#Merge Children and Babies to boolean
for i in raw_data.index:
    if raw_data['children'][i] > 0 or raw_data['babies'][i] > 0:
        raw_data['children'][i] = True
    else:
        raw_data['children'][i] = False

raw_data['children'] = raw_data['children'].astype(bool)

raw_data = raw_data.rename(columns={"children": "has_children"})
raw_data = raw_data.drop(columns=['babies'])

## Map assigned roomtype different from reserved to boolean
temp_assigned_col = raw_data['assigned_room_type']
raw_data['assigned_room_type'] = raw_data['assigned_room_type'].astype(bool)

for i in raw_data.index:
    if temp_assigned_col[i] == raw_data['reserved_room_type'][i]:
        raw_data['assigned_room_type'][i] = True
    else:
        raw_data['assigned_room_type'][i] = False


raw_data = raw_data.rename(columns={"assigned_room_type": "assigned_room_type_as_reserved"})
raw_data = raw_data.drop(columns=['reserved_room_type'])
del temp_assigned_col

# Merge total nights of stay
for i in raw_data.index:            
    raw_data['stays_in_weekend_nights'][i] += raw_data['stays_in_week_nights'][i]

raw_data = raw_data.rename(columns={"stays_in_weekend_nights": "total_nights"})
raw_data = raw_data.drop(columns=['stays_in_week_nights'])

# Remove day and week, keep year and turn month into sin and cos

helper_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}

# Using sin and cos because if only one function is used (only sin or only cos), then two different months can have the same value
raw_data = raw_data.rename(columns={"arrival_date_week_number": "arrival_month_sin", "arrival_date_day_of_month" : "arrival_month_cos"})

raw_data['arrival_month_sin'] = raw_data['arrival_month_sin'].astype(float)
raw_data['arrival_month_cos'] = raw_data['arrival_month_cos'].astype(float)

for i in raw_data.index:
    raw_data['arrival_month_sin'][i] = np.sin(2*np.pi*helper_dict[raw_data['arrival_date_month'][i]]/12)
    raw_data['arrival_month_cos'][i] = np.cos(2*np.pi*helper_dict[raw_data['arrival_date_month'][i]]/12)

raw_data = raw_data.drop(columns=['arrival_date_month'])

# Remove adr, since it has no value for the data mining
raw_data = raw_data.drop(columns=['adr'])
raw_data = raw_data.drop(columns=['reservation_status_date'])

C:\Users\MaxBax\AppData\Local\Temp\ipykernel_14892\1865022957.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['lead_time'][i] = "moderate"
C:\Users\MaxBax\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MaxBax\AppData\Local\Temp\ipykernel_14892\1865022957.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [51]:
## Add Missing Values

for i in raw_data.index:
    if pd.isna(raw_data['agent'][i]):
        raw_data['agent'][i] = "None"
    if pd.isna(raw_data['company'][i]):
        raw_data['company'][i] = "None"


In [52]:
raw_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_month_sin,arrival_month_cos,total_nights,adults,has_children,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,assigned_room_type_as_reserved,booking_changes,deposit_type,agent,company,was_in_waiting_list,customer_type,required_car_parking_spaces,has_special_requests,reservation_status
63,Resort Hotel,False,moderate,2015,-0.5,-0.866025,3,2,False,BB,IRL,Offline TA/TO,TA/TO,False,0,0,False,0,No Deposit,115.0,None,False,Contract,False,False,Check-Out
74,Resort Hotel,True,short-term,2015,-0.5,-0.866025,2,3,False,BB,PRT,Online TA,TA/TO,False,0,0,True,0,No Deposit,242.0,None,False,Transient,False,False,Canceled
122,Resort Hotel,True,short-term,2015,-0.5,-0.866025,1,1,False,BB,PRT,Direct,Direct,False,0,0,False,1,No Deposit,250.0,None,False,Transient,False,False,No-Show
343,Resort Hotel,True,short-term,2015,-0.5,-0.866025,5,2,False,BB,PRT,Online TA,TA/TO,False,0,0,True,0,No Deposit,241.0,None,False,Transient,False,False,No-Show
354,Resort Hotel,False,very-short-term,2015,-0.5,-0.866025,2,4,False,BB,PRT,Direct,Direct,False,0,0,False,0,No Deposit,None,None,False,Transient,False,False,Check-Out


In [53]:
# Check for Null values
raw_data.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_month_sin                 0
arrival_month_cos                 0
total_nights                      0
adults                            0
has_children                      0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
assigned_room_type_as_reserved    0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
was_in_waiting_list               0
customer_type                     0
required_car_parking_spaces       0
has_special_requests              0
reservation_status                0
dtype: int64

In [54]:
raw_data.to_csv('../data/preprocessed_data.csv', index=False)